In [ ]:
using ReactiveMP, Distributions, Random, Rocket, GraphPPL

In [ ]:
a, b = 0.1, -1000
linear_fn(x) = a*x + b
linear_fn_inverse(x) = (1/a)*(x - b)

@model function nonlinear_estimation(n)
    
    m ~ NormalMeanVariance(10, 1)
    τ ~ Gamma(1.1, 1000.1)
    x = randomvar(n)    
    transformed = randomvar(n)
    y = datavar(Float64, n)
    
    for i in 1:n
        x[i] ~ NormalMeanPrecision(m, τ)
        transformed[i] ~ linear_fn(x[i]) where { meta = LinearApproximationKnownInverse(linear_fn_inverse)}
        y[i] ~ NormalMeanPrecision(transformed[i], 0.1)
    end
    
    return transformed, m, y
end

In [ ]:
n = 200

X = rand(Normal(0.0, 1), n)
data = map(x -> linear_fn(x), X);

result = inference(
    model = Model(nonlinear_estimation, n),
    data = (y = data, ),
    initmarginals = (
        m = vague(NormalMeanVariance),
        τ = vague(Gamma),
    ),
    constraints = cs,
    returnvars = (
        m = KeepEach(),
        τ = KeepEach()
    ),
    iterations = 1000,  
    showprogress = true
)